In [1]:
import os
import numpy as np
import cv2
from scipy.stats import kurtosis, skew
from scipy import ndimage
import skimage.measure

In [2]:
def generateDataForImages():
    """
    In this function, various techniques for augementing image data, such as random rotations
    and translations, zooming on images will performed to produce a dataset of size ~1600 images.
    RETURN :
        a list of 2-tuples where the first element in each tuple is an image's numpy 2-D array 
        and the second label is the corresponding label, 1 for real note and 0 for fake note
    """
    
    # this will be the function used to resize and grayscale the raw input image

    def resizeGrayCenter(img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (600, 300), interpolation = cv2.INTER_AREA)
        img = img - img.mean()
        return img
    
 
    # a list of 2-tuples to hold an image's numpy 2-D array 
    # and the corresponding label, 1 for real and 0 for fake
    trainingData = []
    noOfRows = 300
    noOfCols = 600
    
    # first all real currency notes' images and then all the fake ones'
    # i am skeptical of the consequences :-/
    label = 1
    
    for directory in ['Real Notes/', 'Duplicate Notes/']:
        for filename in os.listdir(directory):
            print (filename)
            img = cv2.imread(directory + filename)
            img = resizeGrayCenter(img)

            # TRANSLATIONS
            # will produce (6 + 6 + 1) x (6 + 6 + 1) = 169 images
            # stride of 5 pixels along both axis along all 4 directions
            for x in range(30, -35, -5):
                for y in range(30, -35, -5):
                    translationMatrix = np.float32([ [1,0,x], [0,1,y] ])
                    imgTrns = cv2.warpAffine(img, translationMatrix, (noOfCols, noOfRows))
                    trainingData.append((np.var(imgTrns),skew(imgTrns,axis=None),kurtosis(imgTrns,axis=None),skimage.measure.shannon_entropy(imgTrns),label))


            # ROTATIONS
            # we produce 41 different angles in the range of -10 to 10
            # with the step being equal to 0.5
            for angle in range(20, -21, -1):
                rotationMatrix = cv2.getRotationMatrix2D((noOfCols/2, noOfRows/2), float(angle)/2, 1)
                imgRotated = cv2.warpAffine(img, rotationMatrix, (noOfCols, noOfRows))
                trainingData.append((np.var(imgRotated),skew(imgRotated,axis=None),kurtosis(imgRotated,axis=None),skimage.measure.shannon_entropy(imgRotated), label))

            # PROJECTIVE TRANSFORMATIONS for ZOOMING IN AND ZOOMING OUT
            # will produce (30 + 30) images for the dataset
            # 1ST ZOOMING IN ...
            for step in np.arange(0.001, 0.031, 0.001):
                srcPoints = np.float32([[int(step*(noOfCols-1)),int(step*(noOfRows-1))], [int((1-step)*(noOfCols-1)),int(step*(noOfRows-1))], [int(step*(noOfCols-1)),int((1-step)*(noOfRows-1))], [int((1-step)*(noOfCols-1)), int((1-step)*(noOfRows-1))]])
                dstPoints = np.float32([[0,0], [noOfCols-1,0], [0,noOfRows-1], [noOfCols-1,noOfRows-1]]) 
                projective_matrix = cv2.getPerspectiveTransform(srcPoints, dstPoints)
                imgZoomed = cv2.warpPerspective(img, projective_matrix, (noOfCols,noOfRows))
                trainingData.append((np.var(imgZoomed),skew(imgZoomed,axis=None),kurtosis(imgZoomed,axis=None),skimage.measure.shannon_entropy(imgZoomed), label))
            # 2ND ZOOMING OUT ...
            for step in np.arange(0.001, 0.031, 0.001):
                srcPoints = np.float32(np.float32([[0,0], [noOfCols-1,0], [0,noOfRows-1], [noOfCols-1,noOfRows-1]]))
                dstPoints = np.float32([[int(step*(noOfCols-1)),int(step*(noOfRows-1))], [int((1-step)*(noOfCols-1)),int(step*(noOfRows-1))], [int(step*(noOfCols-1)),int((1-step)*(noOfRows-1))], [int((1-step)*(noOfCols-1)), int((1-step)*(noOfRows-1))]]) 
                projective_matrix = cv2.getPerspectiveTransform(srcPoints, dstPoints)
                imgZoomed = cv2.warpPerspective(img, projective_matrix, (noOfCols,noOfRows))
                trainingData.append((np.var(imgZoomed),skew(imgZoomed,axis=None),kurtosis(imgZoomed,axis=None),skimage.measure.shannon_entropy(imgZoomed), label))
        # set label for fake images to come
        label = 0
            
    return trainingData

In [3]:
imagenlabel=generateDataForImages()
len(imagenlabel)


2k1.jpg
2k2.jpg
2k3.jpg
2k4.jpg
500 (1).jpg
500 (10).jpg
500 (11).jpg
500 (12).jpg
500 (13).jpg
500 (2).jpg
500 (3).jpg
500 (4).jpg
500 (5).jpg
500 (6).jpg
500 (7).jpg
500 (8).jpg
500 (9).jpg
2kd1.jpg
2kd2.jpg
2kd3.jpg


5400

In [4]:
from numpy.random import shuffle
imagenlabel=5*imagenlabel
shuffle(imagenlabel)
imagenlabel

[(1308.611221349671,
  -0.007764335532723447,
  -0.6322742305327624,
  7.09332710266719,
  1),
 (1381.959402230726,
  -1.4495826806037326,
  3.101753882612786,
  15.228939502374397,
  0),
 (1435.3894955379005,
  -0.4624616478668985,
  -0.598459463904975,
  6.969583917390975,
  1),
 (1232.5240442513307,
  -1.6985537020206922,
  4.383114981677282,
  6.592073163267719,
  0),
 (1363.2710711400798,
  -0.5007548754948102,
  -0.5202245152048008,
  15.255952881130142,
  1),
 (1431.71484081968,
  -1.5815368535537981,
  3.352120241238537,
  6.7809887971376615,
  0),
 (1203.3300406227315,
  -0.2043699461651836,
  -0.008108021119575426,
  6.699252480659949,
  1),
 (1147.0319831382353,
  -0.3911529462002835,
  -0.01018741736611739,
  6.818048922537129,
  1),
 (1703.7849455907358,
  -0.4232343474605919,
  -0.498714274252277,
  15.190221458749306,
  1),
 (675.3500366504938,
  -0.6261053705419626,
  0.47406347663627413,
  6.6518371460670895,
  1),
 (1186.3337179428206,
  -1.591543125540587,
  3.826929

In [5]:
import csv

In [6]:
with open('dataset.csv', 'w') as f: 
    write = csv.writer(f)  
    write.writerows(imagenlabel) 